# Managing Information Loss with xarray operations

Sometimes, you can lose important information from your dataset when performing operations.
You will likely want to keep track of the attributes, `nodata`, and `CRS`.

API Reference:

- [rio.to_raster()](../rioxarray.rst#rioxarray.rioxarray.RasterDataset.to_raster)
- [rio.write_crs()](../rioxarray.rst#rioxarray.rioxarray.XRasterBase.write_crs)
- [rio.update_attrs()](../rioxarray.rst#rioxarray.rioxarray.XRasterBase.update_attrs)
- [rio.crs](../rioxarray.rst#rioxarray.rioxarray.XRasterBase.crs)
- [rio.nodata](../rioxarray.rst#rioxarray.rioxarray.RasterArray.nodata)

In [1]:
import rioxarray

See docs for [rioxarray.open_rasterio](../rioxarray.rst#rioxarray-open-rasterio)

In [2]:
rds = rioxarray.open_rasterio(
    "../../test/test_data/input/PLANET_SCOPE_3D.nc",
    variable=["green"],
)

Notice the original data:

In [3]:
rds.green.attrs, rds.green.rio.crs, rds.green.rio.nodata

({'grid_mapping': 'spatial_ref',
  'nodata': 0,
  'units': ('DN', 'DN'),
  '_FillValue': nan,
  'transform': (3.0, 0.0, 466266.0, 0.0, -3.0, 8084700.0),
  'scale_factor': 1.0,
  'add_offset': 0.0},
 CRS.from_epsg(32722),
 nan)

Notice how information is lost in the operation:

In [4]:
new_ds = rds.green + rds.green
new_ds.attrs, new_ds.rio.crs, new_ds.rio.nodata

({}, CRS.from_epsg(32722), None)

The solution is to save the original attributes and then copy them over
once the operation is complete:

In [5]:
new_ds = rds.green + rds.green
new_ds.rio.write_crs(rds.green.rio.crs, inplace=True)
new_ds.rio.update_attrs(rds.green.attrs, inplace=True)
new_ds.attrs, new_ds.rio.crs, new_ds.rio.nodata

({'grid_mapping': 'spatial_ref',
  'nodata': 0,
  'units': ('DN', 'DN'),
  '_FillValue': nan,
  'transform': (3.0, 0.0, 466266.0, 0.0, -3.0, 8084700.0),
  'scale_factor': 1.0,
  'add_offset': 0.0},
 CRS.from_epsg(32722),
 nan)

In [6]:
new_ds.rio.to_raster("combination_keep_attrs.tif")

In [7]:
!rio info combination_keep_attrs.tif

{"bounds": [466266.0, 8084670.0, 466296.0, 8084700.0], "colorinterp": ["gray", "undefined"], "count": 2, "crs": "EPSG:32722", "descriptions": ["green", "green"], "driver": "GTiff", "dtype": "float64", "height": 10, "indexes": [1, 2], "interleave": "pixel", "lnglat": [-51.31732641226951, -17.322997474192466], "mask_flags": [["nodata"], ["nodata"]], "nodata": NaN, "res": [3.0, 3.0], "shape": [10, 10], "tiled": false, "transform": [3.0, 0.0, 466266.0, 0.0, -3.0, 8084700.0, 0.0, 0.0, 1.0], "units": [null, null], "width": 10}
